We start by adding the necessary folders to the current working path.

In [ ]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add HyperbolicsLCS folder to current working path
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/HyperbolicLCS")

# Overview

In the following notebook we extract hyperbolic LCS from tensorlines of the eigenvector field associated to the minimum/maximum eigenvalue of the Cauchy Green strain tensor a 2-D homogeneous, isotropic turbulence. The notebook is structured as follows:

1. Import velocity data stored in the folder 'data/Turbulence'. Make sure that all the necessary files are stored in the folder 'data/Turbulence'.
<br />
2. Define computational parameters (such as the number of cores) and data.
 <br />
3. Define spatio-temporal domain.
 <br />
4. Interpolate velocity from (discrete) gridded data.
 <br />
5. Hyperbolic LCS from forward computation:

    * Compute gradient of flow map $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $ over meshgrid using an auxiliary grid. 
    <br />
    
    * Compute Cauchy Green (CG) strain tensor as: 
    <br />
    
    \begin{equation} \mathbf{C}_{t_0}^{t_N}(\mathbf{x}_0) = \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0)^T\mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0). \end{equation}
   <br />
   
    * Compute maximum eigenvalues $ \lambda_2(\mathbf{x}_0) $ of $ \mathbf{C}_{t_0}^{t_N}(\mathbf{x}_0) $.
    <br />
    
    * Compute repelling LCS (extracted over the time-interval $ [t_0, t_N] $) at time $ t_0 $ as tensorlines tangent to the eigenvector field $ \mathbf{\xi_1(\mathbf{x})} $ associated to the minimum eigenvalue $ \lambda_1(\mathbf{x}) $ of $ \mathbf{C}_{t_0}^{t_N}(\mathbf{x}_0) $:
    <br />
    
    \begin{equation}
    \dfrac{d}{ds} \mathbf{x}(s) =  \mathbf{\xi_1(\mathbf{x})}
    \end{equation}
    <br />
    
    For repelling LCS, the tensorlines are launched from local maxima of the $ \lambda_2(\mathbf{x}) $-field.
    <br />
    
    * Compute attracting LCS (extracted over the time-interval $ [t_0, t_N] $) at time $ t_0 $ as tensorlines tangent to the eigenvector field $ \mathbf{\xi_2(\mathbf{x})} $ associated to the maximum eigenvalue $ \lambda_2(\mathbf{x}) $ of $ \mathbf{C}_{t_0}^{t_N}(\mathbf{x}_0) $:
    <br />
    
    \begin{equation}
    \dfrac{d}{ds} \mathbf{x}(s) =  \mathbf{\xi_2(\mathbf{x})}
    \end{equation}
    <br />
    
    For attracting LCS, the tensorlines are launched from local minima of the $ \lambda_1(\mathbf{x}) $-field.
    <br />
    
    In case of an incompressible flow (as here) local maxima of the $ \lambda_2(\mathbf{x}) $-field coincide with local minima of the $ \lambda_1({\mathbf{x}}) $-field as $ \lambda_2(\mathbf{x}) = \dfrac{1}{\lambda_1(\mathbf{x})} $. 
    <br />
    
    Note that we compute repelling and attracting LCS from a single forward computation. Similarly repelling and attracting LCS can be computed from a single backward computation. Due to the unavoidable orientational discontinuities of the eigenvector-field, the integration has to be performed manually by checking the orientation of the eigenvector at every integration-step. If the orientation is not equivalent to the orientation of the eigenvector at the previous integration step, then the eigenvector has to be flipped before integrating.
 <br />
7. References

# Import data

In [ ]:
# import scipy
import scipy.io

# import numpy
import numpy as np

# load velocity data
turb_u = parent_directory + r'/data/Turbulence/turb_u'
      
# iterate over all files in folder 'turb_u' and load velocity data
i = 0
U = np.zeros((1024, 1024, 251)) 
V = np.zeros((1024, 1024, 251))
for filename in sorted(os.listdir(turb_u)):
    if filename.endswith(".mat"):
        file_u = os.path.join(turb_u, filename)
        mat_turb_u = scipy.io.loadmat(file_u)
        U[:,:,i] = mat_turb_u['u1']
        V[:,:,i] = mat_turb_u['u2']
        i+=1

# load x/y grid
X = scipy.io.loadmat(parent_directory + r'/data/Turbulence/grid.mat')['x']
Y = X.copy().transpose()

# Computational parameters and data

Here we define the computational parameters and the data.

In [ ]:
# Number of cores for parallel computing
Ncores = 16

# time-interval over which data is defined
time_data = np.linspace(0, 25, 251, endpoint = True).reshape(1, -1)

# time resolution of data
dt_data = time_data[0,1]-time_data[0,0]

# periodic boundary conditions
periodic_x = True
periodic_y = True
periodic_t = False
periodic = [periodic_x, periodic_y, periodic_t]

# unsteady velocity field
bool_unsteady = True

# defined domain
# This array denotes where the velocity field is defined (=1) and where it is not defined (=0)
defined_domain = np.isfinite(U[:,:,0]).astype(int)

## resolution of meshgrid
dx_data = X[0,1]-X[0,0]
dy_data = Y[1,0]-Y[0,0]

delta = [dx_data, dy_data]

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [ ]:
# Initial time
t0 = 0 # float

# Final time
tN = 20 # float

# Time step-size
dt = 0.1 # float

# NOTE: For computing the backward trajectories set: tN < t0 and dt < 0.

time = np.arange(t0, tN+dt, dt) # shape (Nt,)

# Length of time interval
lenT = abs(tN-t0) # float

# resolution
Nx = 400
Ny = 400

# set x-limits
xmin = 3
xmax = 5

# set x-limits
ymin = 1
ymax = 3

# specify domain over which to compute LCS
x_domain = np.linspace(xmin, xmax, Nx)
y_domain = np.linspace(ymin, ymax, Ny)

# make sure that domain is part of the data domain.
assert np.min(X) <= xmin <= np.max(X), " xmin must be between "+f'{np.min(X)} and {np.max(X)}'
assert np.min(X) <= xmax <= np.max(X), " xmax must be between "+f'{np.min(X)} and {np.max(X)}'
assert np.min(Y) <= ymin <= np.max(Y), " ymin must be between "+f'{np.min(Y)} and {np.max(Y)}'
assert np.min(Y) <= ymax <= np.max(Y), " ymax must be between "+f'{np.min(Y)} and {np.max(Y)}'
assert np.min(time_data) <= t0 <= np.max(time_data), " t0 must be between "+f'{np.min(time_data)} and {np.max(time_data)}'
assert np.min(time_data) <= tN <= np.max(time_data), " tN must be between "+f'{np.min(time_data)} and {np.max(time_data)}'

# grid-spacing
dx = x_domain[1]-x_domain[0]
dy = y_domain[1]-y_domain[0]

#compute meshgrid
X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

# Define ratio of auxiliary grid spacing vs original grid_spacing
aux_grid_ratio = .1 # float between [1/100, 1/5]
aux_grid = [aux_grid_ratio*dx, aux_grid_ratio*dy] # list (2, )

# Interpolate velocity

In order to evaluate the velocity field at arbitrary locations and times, we interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". Default value is "cubic".

In [ ]:
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, method = "cubic")

Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object

# Hyperbolic LCS from forward computation

## Cauchy Green (CG) strain tensor

The Cauchy Green strain tensor $ C_{t_0}^t(\mathbf{x}) $ is computed by using an auxiliary meshgrid.

In [ ]:
# Import function to compute gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# Import function to compute Cauchy-Green strain tensor
from ipynb.fs.defs.CauchyGreen import CauchyGreen

# Import package for parallel computing
from joblib import Parallel, delayed

# Split x0, y0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

def parallel_C(x0_batch, y0_batch):
    
    # Initial conditions
    X0 = np.array([x0_batch, y0_batch]) # array (2, Nx*Ny)

    # Compute gradient of flow map
    gradFmap = gradient_flowmap(time, X0, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, aux_grid) # shape (N, Nx*Ny, 2, 2)

    # extract gradient from t0 to tN
    gradFmap_t0_tN = gradFmap[-1,:, :, :] # array (N, 2, 2, Nx*Ny)

    # compute CG
    C = gradFmap_t0_tN.copy()*np.nan
    for i in range(gradFmap_t0_tN.shape[2]):
        C[:,:,i] = CauchyGreen(gradFmap_t0_tN[:,:,i]) # array (2, 2, Nx*Ny)
    
    return C

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny, )
x0 = X_domain.ravel() # array (Nx*Ny,)
y0 = Y_domain.ravel() # array (Nx*Ny,)

x0_batch = list(split(x0, Ncores)) # list (Nx*Ny,)
y0_batch = list(split(y0, Ncores)) # list (Nx*Ny,)

results = Parallel(n_jobs=Ncores, verbose = 1)(delayed(parallel_C)(x0_batch[i], y0_batch[i]) for i in range(len(x0_batch)))

# extract CG tensorfield from results of parallel computing
C = results[0]

for res in results[1:]:
    C = np.append(C, res, axis = 2)
    
# reshape array from vectorized form to structured meshgrid
C = C.reshape((2, 2, X_domain.shape[0], X_domain.shape[1])) # array (2, 2, Ny, Nx)

## Eigenvalues/Eigenvectors of Cauchy Green strain tensor

We now compute the eigenvalues and eigenvectors of Cauchy Green strain tensor. We additionally also compute the $ \mathrm{FTLE}_{t_0}^{t_N} $ as we seek to later compare the repelling LCS with features of the $ \mathrm{FTLE} $-field.

In [ ]:
# Import function to compute eigenvalues/eigenvectors
from ipynb.fs.defs.eigen import eigen

# Import function to check location of particle
from ipynb.fs.defs.check_location import check_location

# add FTLE folder to current working path
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/FTLE2D")

# Import function to calculate FTLE from Cauchy-Green strain tensor
from ipynb.fs.defs.FTLE import _FTLE_C

# maximum eigenvalues of CG-tensor
eig1 = C[0,0,:,:].copy()*np.nan # array (Ny, Nx)
# minimum eigenvalues of CG-tensor
eig2 = C[0,0,:,:].copy()*np.nan # array (Ny, Nx)
# eigenvectors associated to maximum eigenvalues of CG-tensor
e1 = np.zeros((eig1.shape[0], eig1.shape[1], 2))*np.nan # array (Ny, Nx, 2)
# eigenvectors associated to minimum eigenvalues of CG-tensor
e2 = np.zeros((eig2.shape[0], eig2.shape[1], 2))*np.nan # array (Ny, Nx, 2)
# FTLE-field
FTLE = C[0,0,:,:].copy()*np.nan # array (Ny, Nx)

#iterate over meshgrid
for i in range(X_domain.shape[0]):
    
    for j in range(Y_domain.shape[1]):
            
        x = [X_domain[i,j], Y_domain[i, j]]
        
        # only compute CG tensor for trajectories starting region where velocity field is defined
        if check_location(X, Y, defined_domain, np.array(x))[0] == "IN":
        
            # compute eigenvalues of CG tensor
            eig1[i,j], eig2[i,j], e1[i,j,:], e2[i,j,:] = eigen(C[:,:,i,j])
            
            # compute FTLE-field from CG tensor
            FTLE[i, j] = _FTLE_C(C[:,:,i,j], lenT)

## Shrinklines (Repelling LCS)

Repelling LCS can be sought among trajectories of the differential equation:

\begin{equation}
\mathbf{x}'_0(s) = \xi_1(\mathbf{x}_0;t_0, t_N),
\label{eq: shrinklines}
\end{equation}

with $ \xi_1 $ denoting the eigenvector associated to the weakest eigenvalue $ \lambda_1 $ of $ C_{t_0}^{t_N}(\mathbf{x}_0) $. The non-orientable vector field is well defiend away from tensorline singularites (points where $ \lambda_1 = \lambda_2 $).
The most repelling shrinklines mark initial positions of repelling LCSs. Repelling LCSs can therefore be located as trajectories of eq. \ref{eq: shrinklines} that have locally the largest averaged $ \lambda_2(\mathbf{x}_0;t_0, t_N) $ among all neighbouring shrinklines.

In [ ]:
# import function to compute tensorlines
from ipynb.fs.defs.tensorlines import _tensorlines_incompressible

# Step-size used for integration
step_size = 0.05 # float

# threshold distance to locate local maxima in the 'eig2'
min_distance = 0.05 # float

# Maximum length of stretchline
max_length = 5 # float

# Number of most relevant tensorlines. If you want all possible tensorlines, then set n_tensorlines = -1
n_tensorlines = 10 # int

# Minimum threshold on rate of attraction of stretchline 
hyperbolicity = 0

# Maximum threshold on number of iterations
n_iterations = 10**3

# Compute shrinklines
shrinklines = _tensorlines_incompressible(X_domain, Y_domain, eig2, e2, min_distance, max_length, step_size, n_tensorlines, hyperbolicity, n_iterations, verbose = False) # list containing stretchlines

# NOTE: 
# If we set verbose = True in _tensorlines_incompressible(), then progress bar is printed.
# Each progress bar represents the computation of a single stretchline starting from a local maximum of lambda_2. 
# If the progress bar does not complete this means that the algorithm converged before reaching its limit on the iterations.
# The algorithm is guaranteed to converge, however, in order to avoid long waiting times we employ a threshold on the number of iterations.

## Stretchlines (Attracting LCS)

Attracting LCS can be sought among trajectories of the differential equation:

\begin{equation}
\mathbf{x}'_0 = \xi_2(\mathbf{x}_0;t_0, t_N),
\label{eq: stretchlines}
\end{equation}

with $ \xi_2 $ denoting the eigenvector associated to the strongest eigenvalue $ \lambda_2 $ of $ C_{t_0}^{t_N}(\mathbf{x}_0) $. The non-orientable vector field is well defiend away from tensorline singularites (points where $ \lambda_1 = \lambda_2 $).
The most attracting stretchlines mark initial positions of attracting LCSs. Attracting LCSs can therefore be located as trajectories of eq. \ref{eq: stretchlines} that have locally the smallest averaged $ \lambda_1(\mathbf{x}_0;t_0, t_N) $ among all neighbouring stretchlines.

In [ ]:
# Step-size used for integration
step_size = 0.05 # float

# threshold distance to locate local maxima in the 'eig2'
min_distance = 0.05 # float

# Maximum length of stretchline
max_length = 5 # float

# Number of most relevant tensorlines. If you want all possible tensorlines, then set n_tensorlines = -1
n_tensorlines = 10 # int

# Minimum threshold on rate of attraction of stretchline 
hyperbolicity = 0
# Maximum threshold on number of iterations
n_iterations = 10**3

# Compute stretchlines
stretchlines = _tensorlines_incompressible(X_domain, Y_domain, eig2, e2, min_distance, max_length, step_size, n_tensorlines, hyperbolicity, n_iterations, verbose = False) # list containing stretchlines

# NOTE: 
# If we set verbose = True in _tensorlines_incompressible(), then progress bar is printed.
# Each progress bar represents the computation of a single stretchline starting from a local maximum of lambda_2. 
# If the progress bar does not complete this means that the algorithm converged before reaching its limit on the iterations.
# The algorithm is guaranteed to converge, however, in order to avoid long waiting times we employ a threshold on the number of iterations.

In [ ]:
############################################ PLOT RESULTS ############################################
import matplotlib.pyplot as plt

# Define figure/axes
fig = plt.figure(figsize = (8, 4), dpi = 600)
ax = plt.axes()

for i in range(len(stretchlines[0])):
    ax.plot(stretchlines[0][i], stretchlines[1][i], c = 'b', linewidth = 1, linestyle = "dashed")
    
for i in range(len(shrinklines[0])):
    ax.plot(shrinklines[0][i], shrinklines[1][i], c = 'r', linewidth = 1, linestyle = "dashed")
    
# add FTLE field to plot
cax = ax.contourf(X_domain, Y_domain, np.ma.masked_invalid(FTLE), levels = 400, cmap = "gist_gray")    

# Set axis limits
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

# Set axis ticks
ax.set_xticks(np.arange(xmin, xmax+0.1, .5))
ax.set_yticks(np.arange(ymin, ymax+0.1, .5))

# Set axis labels
ax.set_xlabel("x", fontsize = 10)
ax.set_ylabel("y", fontsize = 10)

# Colorbar
cbar = fig.colorbar(cax, ticks = np.arange(0, 1, 0.05))

# Plot legend
import matplotlib.patches as mpatches
attracting = mpatches.Patch(color='b', label='attracting LCS')
repelling = mpatches.Patch(color='r', label='repelling LCS')
plt.legend(handles=[attracting, repelling], fontsize = 5,  loc = "lower left")

# Title
ax.set_title("ATTRACTING/REPELLING LCS over forward FTLE", fontsize = 10)
plt.show();

Similarly, we can obtain both repelling and attracting LCS at time $ t_N $ from a single backward computation of the backward flow map $ \mathbf{F}_{t_N}^{t_0}(\mathbf{x}_0) $. Obtaining positions of repelling LCSs at time $ t \in [t_0, t_N] $ from the forward forward computation remains problematic as it requires the forward advection of a discrete approximation of the repelling LCSs under the flow map $ \mathbf{F}_{t_0}^{t}(\mathbf{x}_0) $, generating exponentially growing errors. An analogous statement holds for attracting LCSs at time $ t_N $ by using backward advection under the backward flow map $ \mathbf{F}_{t_N}^{t}(\mathbf{x}_0) $.

Compared to the [$ \mathrm{FTLE} $](../../FTLE2D/Main/FTLETurbulence.ipynb), which is only a diagnostic quantity used for visualizing possible hyperbolic LCSs, the repelling/attracting LCSs derived from the tensorline computation are precise indicators of hyperbolic LCSs. As expected repelling LCS (red solid lines) match well with ridges of the $ \mathrm{FTLE} $ field. Furthermore, due to the orthogonality of the two eigenvector fields $ \mathbf{\xi}_1 $ and $ \mathbf{\xi}_2 $, repelling LCS need always be orthogonal to attracting LCS. Intersections of repelling and attracting LCSs are referred to as generalized finite-time saddles[1, 2, 3].

# References

[1] Farazmand, M., & Haller, G. (2012). Computing Lagrangian coherent structures from their variational theory. Chaos: An Interdisciplinary Journal of Nonlinear Science, 22(1), 013128.

[2] Farazmand, M., & Haller, G. (2013). Attracting and repelling Lagrangian coherent structures from a single computation. Chaos: An Interdisciplinary Journal of Nonlinear Science, 23(2), 023101.

[3] Onu, K., Huhn, F., & Haller, G. (2015). LCS Tool: A computational platform for Lagrangian coherent structures. Journal of Computational Science, 7, 26-36.

[4] Notebook 5.3.2 in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.